# interface: 從協定到 ABC
## 抽象類別代表介面
* 代表 duck typeing 的動態協定，到使介面更明確，並驗證實作一致的抽象基礎類別(ABC)
* duck typeing 協定他是普通介面思維方式是 ABC 法定程式(formality) 與型態檢查


1. 介紹 duck typeing 動態特質
2. 專門討論 ABC 從常見用途當實作介面將他們當成超類別
3. 何時 ABC 會檢查子類別是否符合他所定義的介面，註冊機制如何壤開發者不需要繼承可以宣稱某類別實作一個介面
4. 編寫 ABC 自動識別符合介面不需要繼承或註冊


## 介面與協定
* 每個類別都有介面包含自己實作或是從類別繼承通用屬性包含特殊方法
* 受保護與私有屬性不是介面一部分
* 公有屬性成為物件一部分，因為資料屬性可以改成實作 getter/setter 邏輯，而不會破壞使用一般obj attr用戶端程式碼
* 補充介面定義：某物件公用方法子集合，在系統可以發揮具體作用（一群方法集合，可以扮演一個角色的介面）

```
#範例9-2
class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>
```

```
# 
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y
```

## 挖掘序列
* 實作序列協定一個方法 `__getitem__`

In [3]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]
F = Foo()
F[1]

10

In [6]:
f = Foo()
for i in f: print(i)

0
10
20


In [7]:
20 in F

True

In [8]:
15 in F 

False

In [9]:
dir(F)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

因為序列協定關係沒有 __iter__與 __contains__ 方法也可以讓 in 運算子工作及迭代物件

# Monkey patching 在執行階段實作協定

In [11]:
# shuffle 函式標準用法
from random import shuffle
l = list(range(10))
shuffle(l)
l

[1, 8, 9, 2, 0, 6, 3, 4, 5, 7]

In [12]:
from random import shuffle
import collections

# namedtuple define Card
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    # 起始化牌堆
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
    # 用 len 計算 list 個數
    def __len__(self):
        return len(self._cards)
    # 從 postition 去找牌花色及數字
    def __getitem__(self, position):
        return self._cards[position]


In [13]:
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [14]:
# Monkey patching
def set_card(deck, position, card):
    deck._cards[position] = card


In [15]:
FrenchDeck.__setitem__ = set_card

In [16]:
shuffle(deck)

In [17]:
deck[:5]

[Card(rank='7', suit='clubs'),
 Card(rank='9', suit='spades'),
 Card(rank='10', suit='spades'),
 Card(rank='K', suit='clubs'),
 Card(rank='J', suit='clubs')]